In [ ]:
import numpy as np
def randomize_binary_network(observed_matrix, preserve_degree=True, iterations=1000, seed=None):
    """
    Randomizes a binary adjacency matrix while preserving its structure.

    
    """
    if seed is not None:
        np.random.seed(seed)
    
    # Ensure the input matrix is binary
    if not np.array_equal(observed_matrix, observed_matrix.astype(bool)):
        raise ValueError("The observed matrix must be binary (0s and 1s).")
    
    # Ensure symmetry for undirected networks
    observed_matrix = np.triu(observed_matrix) + np.triu(observed_matrix, 1).T
    randomized_matrix = observed_matrix.copy()

    if preserve_degree:
        # Get all edges (i, j) where i < j (upper triangular matrix)
        edges = np.array(np.transpose(np.triu_indices_from(observed_matrix, k=1)))

        # Perform edge swaps to randomize the network
        for _ in range(iterations):
            # Select two random edges to swap
            idx1, idx2 = np.random.choice(len(edges), 2, replace=False)
            a, b = edges[idx1]
            c, d = edges[idx2]

            # Ensure no duplicate edges or self-loops after rewiring
            if len({a, b, c, d}) == 4 and randomized_matrix[a, d] == 0 and randomized_matrix[c, b] == 0:
                # Swap edges: (a, b) with (c, d) -> (a, d) and (c, b)
                randomized_matrix[a, b] = 0
                randomized_matrix[b, a] = 0
                randomized_matrix[c, d] = 0
                randomized_matrix[d, c] = 0

                randomized_matrix[a, d] = 1
                randomized_matrix[d, a] = 1
                randomized_matrix[c, b] = 1
                randomized_matrix[b, c] = 1

                # Update the edge list
                edges[idx1] = [a, d]
                edges[idx2] = [c, b]
    else:
        # Completely randomize connections while preserving density
        num_edges = np.sum(np.triu(observed_matrix, k=1))  # Count total connections
        randomized_matrix.fill(0)
        
        possible_edges = np.transpose(np.triu_indices_from(observed_matrix, k=1))
        selected_edges = possible_edges[np.random.choice(len(possible_edges), num_edges, replace=False)]
        
        for edge in selected_edges:
            i, j = edge
            randomized_matrix[i, j] = 1
            randomized_matrix[j, i] = 1  # Keep symmetry

    return randomized_matrix


In [ ]:

def generate_random_correlation_matrix(observed_corr_matrix, seed=None):
    """
    Generates a randomized correlation matrix using the HQS algorithm.
   
    """
    if seed is not None:
        np.random.seed(seed)
    
    # Ensure the observed matrix is symmetric and positive semi-definite
    observed_corr_matrix = (observed_corr_matrix + observed_corr_matrix.T) / 2
    eigenvalues, eigenvectors = np.linalg.eigh(observed_corr_matrix)
    
    # Generate random eigenvalues that preserve the original spectrum's range and sum
    random_eigenvalues = np.random.permutation(eigenvalues)
    
    # Reconstruct the random covariance matrix
    random_cov_matrix = eigenvectors @ np.diag(random_eigenvalues) @ eigenvectors.T
    
    # Normalize to form a correlation matrix
    diag_sqrt = np.sqrt(np.diag(random_cov_matrix))
    random_corr_matrix = random_cov_matrix / np.outer(diag_sqrt, diag_sqrt)
    
    # Ensure symmetry (numerical errors may cause slight asymmetry)
    random_corr_matrix = (random_corr_matrix + random_corr_matrix.T) / 2
    
    # Clip values to ensure they're within the valid range for correlations
    np.fill_diagonal(random_corr_matrix, 1.0)  # Enforce 1 on the diagonal
    random_corr_matrix = np.clip(random_corr_matrix, -1, 1)
    
    return random_corr_matrix

In [9]:
def PercentageThresholding(correlationMatrix, percentageToKeep):
    '''
    Function to apply a percentage based thresholding, returns new adjacency matrix
    '''
    
    # Apply Fisher z-transformation to the correlation matrix
    transformedMatrix = np.arctanh(correlationMatrix)
    
    # Flatten the matrix, sort the values in descending order
    sortedValues = np.sort(transformedMatrix.flatten())[::-1]
    
    # Determine the thresholded value based on percentage (how many elements to keep)
    numElementsToKeep = int(len(sortedValues) * percentageToKeep)
    thresholdValue = sortedValues[numElementsToKeep - 1]
    
    # Create a new adjacency matrix based on our thresholding
    adjMatrix = np.zeros(transformedMatrix.shape, dtype=int) # int for binary
    adjMatrix[transformedMatrix >= thresholdValue] = 1 # 1 if keeping, 0 if thresholded
    

    
    return adjMatrix

In [ ]:
import numpy as np

def generate_random_correlation_matrix_from_observed(C_obs):
    # Step 1: Extract the size of the matrix
    N = C_obs.shape[0]
    
    # Step 2: Calculate the mean (e) and variance (v) of the off-diagonal elements
    off_diagonal_elements = []
    for i in range(N):
        for j in range(i + 1, N):
            off_diagonal_elements.append(C_obs[i, j])

    e = np.mean(off_diagonal_elements)  # Mean of off-diagonal elements
    v = np.var(off_diagonal_elements)  # Variance of off-diagonal elements
    
    # Step 3: Generate a random covariance matrix using the HQS algorithm
    # Initialize an empty matrix to store the HQS result
    C_HQS = np.zeros_like(C_obs)
    
    # Step 4: Assign values to the off-diagonal elements
    for i in range(N):
        for j in range(i + 1, N):
            # Randomly generate values around the mean with the calculated variance
            C_HQS[i, j] = np.random.normal(e, np.sqrt(v))
            C_HQS[j, i] = C_HQS[i, j]  # Ensure symmetry
    
    # Step 5: Set the diagonal elements to 1 (correlations are normalized)
    np.fill_diagonal(C_HQS, 1)
    
    # Step 6: Normalize the resulting correlation matrix (if needed)
    

    return C_HQS

In [ ]:
# import networkx as nx
# import copy
# file_path = R'C:\GIT\Connectomics\Research_Project\Binary_Notebooks\Binary_Output\10%\Matrices\Caltech_0051456_rois_cc400_mat.txt'

# observed_matrix = np.loadtxt(file_path)


# import numpy as np

# def get_off_diagonal_elements(corr_matrix):
#     """
#     Extract the mean and variance of the off-diagonal elements.
#     """
#     N = corr_matrix.shape[0]
#     off_diagonal_elements = []
    
#     for i in range(N):
#         for j in range(N):
#             if i != j:
#                 off_diagonal_elements.append(corr_matrix[i, j])

#     mean_off_diagonal = np.mean(off_diagonal_elements)
#     variance_off_diagonal = np.var(off_diagonal_elements)
    
#     return mean_off_diagonal, variance_off_diagonal

# def generate_gaussian_samples(N, mu, sigma_squared):
#     """
#     Generate a Gaussian sample matrix with mean `mu` and variance `sigma_squared`.
#     """
#     samples = np.random.normal(loc=mu, scale=np.sqrt(sigma_squared), size=(N, N))
#     return samples

# def normalize_to_correlation_matrix(cov_matrix):
#     """
#     Normalize the covariance matrix to a correlation matrix.
#     """
#     N = cov_matrix.shape[0]
#     correlation_matrix = cov_matrix.copy()

#     for i in range(N):
#         for j in range(N):
#             if i == j:
#                 correlation_matrix[i, j] = 1
#             else:
#                 correlation_matrix[i, j] = cov_matrix[i, j] / np.sqrt(cov_matrix[i, i] * cov_matrix[j, j])

#     return correlation_matrix

# def hqs_algorithm(observed_corr_matrix):
#     """
#     Generate a random correlation matrix using the Hirschberger-Qi-Steuer (HQS) algorithm.
#     """
#     # Step 1: Extract the mean and variance of the off-diagonal elements
#     mean_off_diagonal, variance_off_diagonal = get_off_diagonal_elements(observed_corr_matrix)
    
#     # Step 2: Calculate m, μ, and σ^2
#     m = max(2, mean_off_diagonal)  # Ensure m is at least 2
#     mu = np.sqrt(mean_off_diagonal / m)
#     sigma_squared = mu**2 + np.sqrt(mu**4 + variance_off_diagonal / m)
    
#     # Step 3: Generate Gaussian samples
#     gaussian_samples = generate_gaussian_samples(observed_corr_matrix.shape[0], mu, sigma_squared)
    
#     # Step 4: Compute the covariance matrix C = XX^T
#     covariance_matrix = np.dot(gaussian_samples, gaussian_samples.T)
    
#     # Step 5: Normalize the covariance matrix to get the correlation matrix
#     correlation_matrix = normalize_to_correlation_matrix(covariance_matrix)
    
#     return correlation_matrix

# # Example usage:
# # Assume `observed_corr_matrix` is a numpy array of shape (N, N)
# # observed_corr_matrix = np.random.rand(5, 5)  # Example: 5 variables

# # np.fill_diagonal(observed_corr_matrix, 1)  # Filling diagonal with 1s for correlation matrix
# random_correlation_matrix = generate_random_correlation_matrix_from_observed(observed_matrix)

# print("Random Correlation Matrix:")
# print(random_correlation_matrix)

# adj = PercentageThresholding(random_correlation_matrix, 0.3)

# r_graph = nx.from_numpy_array(adj)

# print(nx.average_clustering(r_graph))


Random Correlation Matrix:
[[ 1.         -0.45118195 -0.24967267 ...  0.09978758 -0.07033198
   0.17735578]
 [-0.45118195  1.          0.54096062 ...  0.95707637 -0.32489243
  -0.1057407 ]
 [-0.24967267  0.54096062  1.         ...  0.15698053  0.15611211
   0.43262027]
 ...
 [ 0.09978758  0.95707637  0.15698053 ...  1.          0.11627885
   0.03510133]
 [-0.07033198 -0.32489243  0.15611211 ...  0.11627885  1.
   0.38796784]
 [ 0.17735578 -0.1057407   0.43262027 ...  0.03510133  0.38796784
   1.        ]]


C:\Users\mattm\AppData\Local\Temp\ipykernel_30248\1786135690.py:7: RuntimeWarning: divide by zero encountered in arctanh
  transformedMatrix = np.arctanh(correlationMatrix)
C:\Users\mattm\AppData\Local\Temp\ipykernel_30248\1786135690.py:7: RuntimeWarning: invalid value encountered in arctanh
  transformedMatrix = np.arctanh(correlationMatrix)


0.2974412037756845


In [ ]:
# Example observed correlation matrix (randomly generated for demonstration)
import networkx as nx
import copy
file_path = R'C:\GIT\Connectomics\Research_Project\Binary_Notebooks\Binary_Output\10%\Matrices\Caltech_0051456_rois_cc400_mat.txt'

observed_matrix = np.loadtxt(file_path)

G_O = nx.from_numpy_array(observed_matrix)

graph = copy.deepcopy(G_O)
# G_R = hqs_adjustment(graph)
G_R = nx.random_reference(graph, 100, connectivity=False)

print(nx.average_clustering(G_R))
arr = nx.to_numpy_array(G_R)

adjMat = PercentageThresholding(arr, 0.1)

new_graph = nx.from_numpy_array(adjMat)

print(nx.average_clustering(new_graph))
# G_R = nx.double_edge_swap(graph, nswap=graph.number_of_edges() * 20, max_tries= graph.number_of_edges() * 200)



0.3256584292176968


C:\Users\mattm\AppData\Local\Temp\ipykernel_30248\1786135690.py:7: RuntimeWarning: divide by zero encountered in arctanh
  transformedMatrix = np.arctanh(correlationMatrix)


1.0


In [7]:
print(nx.degree_centrality(G_R))


print(nx.global_efficiency(G_O))
print(nx.global_efficiency(G_R))

print("Avg Clustering")
print(nx.average_clustering(G_O))
print(nx.average_clustering(G_R))

{0: 0.04859335038363172, 1: 0.13043478260869568, 2: 0.24040920716112535, 3: 0.0025575447570332483, 4: 0.2659846547314578, 5: 0.010230179028132993, 6: 0.05882352941176471, 7: 0.02813299232736573, 8: 0.2659846547314578, 9: 0.23785166240409208, 10: 0.023017902813299233, 11: 0.08184143222506395, 12: 0.012787723785166242, 13: 0.015345268542199489, 14: 0.04347826086956522, 15: 0.07161125319693096, 16: 0.04092071611253197, 17: 0.02813299232736573, 18: 0.13043478260869568, 19: 0.2225063938618926, 20: 0.18925831202046037, 21: 0.2276214833759591, 22: 0.15601023017902815, 23: 0.03580562659846548, 24: 0.01790281329923274, 25: 0.18158567774936063, 26: 0.05370843989769821, 27: 0.020460358056265986, 28: 0.2480818414322251, 29: 0.04347826086956522, 30: 0.0, 31: 0.07672634271099744, 32: 0.0, 33: 0.23529411764705885, 34: 0.030690537084398978, 35: 0.05370843989769821, 36: 0.005115089514066497, 37: 0.21994884910485935, 38: 0.1739130434782609, 39: 0.04347826086956522, 40: 0.2122762148337596, 41: 0.00511508